In [ ]:
OUTPUT_DATASET_ID = 'cropped-cats-and-dogs'
OUTPUT_DATASET_NAME = 'Cropped Cats and Dogs'
OUTPUT_PATH = './output'

YOLO_V3_PATH = '../input/yolov3-files'
INPUT_FILES = 'train/cat.*.jpg'

# kaggle_secrets not supported by Google Cloud Platform for Kaggle(Beta) at this time
# from kaggle_secrets import UserSecretsClient
# user_secrets = UserSecretsClient()
#API_TOKEN = user_secrets.get_secret("Crop Cats and Cogs YOLOv3")

USER_ID = 'KAGGLE-USERNAME' # use your own username
API_TOKEN = 'KAGGLE-API-TOKEN' # use your own kaggle api key

# same size is used in Augment Cats and Dogs
X_SIZE = 224
Y_SIZE = 224

use_gpu = True
show = False
save = True

confidence = 0.5
threshold = 0.3
max_size = 128

run_limit = 10

In [ ]:
# unzip data

import zipfile

with zipfile.ZipFile("../input/dogs-vs-cats-redux-kernels-edition/train.zip","r") as z:
    z.extractall(".")
    
with zipfile.ZipFile("../input/dogs-vs-cats-redux-kernels-edition/test.zip","r") as z:
    z.extractall(".")

This section was inspired by: [YOLO Object Detection with OpenCV](https://gilberttanner.com/blog/yolo-object-detection-with-opencv) by [Gilbert Tanner](https://gilberttanner.com/)

In [ ]:
# Crop Images

import os
import sys
import cv2
import glob
import time
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

weights_path = YOLO_V3_PATH + '/yolov3.weights'
config_path = YOLO_V3_PATH + '/yolov3.cfg'
labels_path = YOLO_V3_PATH + '/coco.names'    
test_image_path = YOLO_V3_PATH + '/image1.jpg'

def extract_boxes_confidences_classids(outputs, confidence, width, height):
    
    boxes = []
    confidences = []
    classIDs = []

    for output in outputs:
        
        for detection in output: 
            
            # Extract the scores, classid, and the confidence of the prediction
            scores = detection[5:]
            classID = np.argmax(scores)
            conf = scores[classID]
            
            # Consider only the predictions that are above the confidence threshold
            if conf > confidence:
                
                # Scale the bounding box back to the size of the image
                box = detection[0:4] * np.array([width, height, width, height])
                centerX, centerY, w, h = box.astype('int')

                # Use the center coordinates, width and height to get the coordinates of the top left corner
                x = int(centerX - (w / 2))
                y = int(centerY - (h / 2))

                boxes.append([x, y, int(w), int(h)])
                confidences.append(float(conf))
                classIDs.append(classID)

    return boxes, confidences, classIDs


def crop_image(image, boxes, confidences, classIDs, idxs, colors, filename, scale):
    
    wasProcessed = False
    
    if len(idxs) > 0:
        
        rgba_image = cv2.cvtColor(image, cv2.COLOR_RGB2RGBA)
        
        for i in idxs.flatten():
            
            if labels[classIDs[i]] == 'cat':
                
                # extract bounding box coordinates
                x, y = boxes[i][0], boxes[i][1]
                w, h = boxes[i][2], boxes[i][3]

                if w < scale or h < scale or x < 0 or y < 0:
                    continue;
                
                wasProcessed = True
                
                img_cropped = rgba_image[y:y+h, x:x+w]
                
                # show the output image
                if show:
                    %matplotlib inline
                    plt.rcParams['figure.figsize'] = (10.0, 10.0)
                    plt.imshow(cv2.cvtColor(img_cropped, cv2.COLOR_BGR2RGB))
                    plt.show()

                fname, ext = os.path.splitext(filename)                   
                cv2.imwrite(OUTPUT_PATH + '/' + fname + '-' + i.astype(str) + ext, img_cropped)

                # scale natively
                img_resized = cv2.resize(img_cropped, (X_SIZE, Y_SIZE), interpolation=cv2.INTER_AREA)
                cv2.imwrite(OUTPUT_PATH + '/' + fname + '-' + i.astype(str) + '-resized' + ext, img_resized)

                # also write black and white and inverted (negative color) images
                gray = cv2.cvtColor(img_resized, cv2.COLOR_BGR2GRAY)
                negative_image = cv2.cvtColor(1 - gray, cv2.COLOR_GRAY2BGR)

                cv2.imwrite(OUTPUT_PATH + '/' + fname + '-' + i.astype(str) + '-resized-neg' + ext, negative_image)
                cv2.imwrite(OUTPUT_PATH + '/' + fname + '-' + i.astype(str) + '-resized-bw' + ext, cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR))

                # also write black and white and inverted (negative color) images
                gray = cv2.cvtColor(img_cropped, cv2.COLOR_BGR2GRAY)
                negative_image = cv2.cvtColor(1 - gray, cv2.COLOR_GRAY2BGR)
                
                cv2.imwrite(OUTPUT_PATH + '/' + fname + '-' + i.astype(str) + '-neg' + ext, negative_image)
                cv2.imwrite(OUTPUT_PATH + '/' + fname + '-' + i.astype(str) + '-bw' + ext, cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR))
                
                return wasProcessed

            
def make_prediction(net, layer_names, labels, image, confidence, threshold):
    
    height, width = image.shape[:2]
    
    # Create a blob and pass it through the model
    blob = cv2.dnn.blobFromImage(image, 1 / 255.0, (416, 416), swapRB=True, crop=False)
    net.setInput(blob)
    outputs = net.forward(layer_names)

    # Extract bounding boxes, confidences and classIDs
    boxes, confidences, classIDs = extract_boxes_confidences_classids(outputs, confidence, width, height)

    # Apply Non-Max Suppression
    idxs = cv2.dnn.NMSBoxes(boxes, confidences, confidence, threshold)

    return boxes, confidences, classIDs, idxs


if __name__ == '__main__':

    print("Cropping Images")
    start_time = time.time()

    i = 0
    
    # Get the labels
    labels = open(labels_path).read().strip().split('\n')

    # Create a list of colors for the labels
    colors = np.random.randint(0, 255, size=(len(labels), 3), dtype='uint8')

    # Load weights using OpenCV
    net = cv2.dnn.readNetFromDarknet(config_path, weights_path)

    if use_gpu:
        net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
        net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)
        print('Using GPU')
        
    if save:
        os.makedirs(OUTPUT_PATH, exist_ok=True)

    # Get the ouput layer names
    layer_names = net.getLayerNames()
    layer_names = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]

    for filepath in glob.iglob(INPUT_FILES):

        image = cv2.imread(filepath)

        boxes, confidences, classIDs, idxs = make_prediction(net, layer_names, labels, image, confidence, threshold)

        # print(filepath)
        filename = os.path.basename(filepath)
        wasProcessed = crop_image(image, boxes, confidences, classIDs, idxs, colors, filename, max_size)

        if wasProcessed:
            i += 1
            
            if i > run_limit:
                break
            
            print(str(i) + ': ' + filename)

    run_time = time.time()-start_time
    print('Done Cropping Images - Elapsed Time: {:.1f}'.format(run_time) + ' Secs')

In [ ]:
! python -m pip install --index-url https://test.pypi.org/simple/ --no-deps kaggle_uploader-screamatthewind

In [ ]:
# Google Cloud Plaform for Kaggle(Beta) does not support /usr/lib modules at this time 
# Save Output Dataset

import time
import os

from kaggle_uploader import kaggle_uploader 

print("Saving Images")
start_time = time.time()

# kaggle_secrets are not supported by Google Cloud Platform for Kaggle(Beta) at this time
# from kaggle_secrets import UserSecretsClient
# user_secrets = UserSecretsClient()
# api_secret = user_secrets.get_secret("Crop Cats and Cogs YOLOv3")

kaggle_uploader.resources = []
kaggle_uploader.init_on_kaggle(USER_ID, API_TOKEN)
kaggle_uploader.base_path = OUTPUT_PATH
kaggle_uploader.title = OUTPUT_DATASET_NAME
kaggle_uploader.dataset_id = OUTPUT_DATASET_ID
kaggle_uploader.user_id = USER_ID

for filename in os.listdir(kaggle_uploader.base_path):
    kaggle_uploader.add_resource(filename, filename)
    
kaggle_uploader.update("new version")

run_time = time.time()-start_time
print('Done Saving Images - Total Time: {:.1f}'.format(run_time) + ' Secs')

# If you get an error during update, it is typically because of an invalid api key, bad username, 
# or the dataset does not exist.  This code does not create datasets.  It updates existing ones
